In [1]:
from src.data.dataset_functions import get_dataframe_onlymods_full

mod_dataframe = get_dataframe_onlymods_full(True)

providing ../data/raw/2022_place_canvas_history-000000000004.csv ...
../data/raw/2022_place_canvas_history-000000000004.csv is already in data/raw
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d6e3b06b-89cc-4d9e-abf3-7f2129cde6ad;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
:: resolution report :: resolve 81ms :: artifacts dl 4ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------

<div style="color:#246fd6">
Im folgenden sollen die Statistiken für Mods bestimmt werden:


1) Wie oft mussten Mods zensieren im jeweiligen Datensatz?

2) Wie viele Pixel wurden insgesamt von einem Mod im jeweiligen Datensatz zensiert?

3) Werden Regionen deutlich, in denen Mods häufiger eingreifen mussten?

    3.1) Wenn ja, welche Regionen und welchen Artworks ist das zugeordnet?

    3.2) Zu bestimmen, welche Pixel mehr als einmal zensiert wurden, also min. 2-mal.

    -> wird von diesem Ansatz her verworfen!!!

    -> Es soll nun ein Canvas erstellt werden, in dem die Quadrate der Mods nachgezeichnet werden sollen in verschiedenen Farben,
    um so evtl. Gebiete direkt optisch darstellen zu können.

4) Evtl.: Wurden von Bots gesetzte Pixel zensiert?

-> viel zu umfangreich & komplex, genauso wie das Nachzeichnen der von Mods zensierten Artworks,
da dies viele Apsekte verücksichtigen muss, wie welche Pixel zuletzt vorher gesetzt wurden etc.

</div>

<div style="color:#246fd6">
Zur Beantwortung der ersten Frage:

Mods sind im "StandardDataFrame" nicht enthalen, nur noch im spezifischen ModsDataFrame!

Um die Frage zu beantworten muss gezählt werden, wie viele Einträge für Mods im jeweiligen Datensatz exitieren.
D.h. auch Duplikate von User_IDs sind relevant, sodass einfach nur gezählt werden muss, wie viele Einträge insgesamt im ModsDataFrame existieren.

Das wäre dann auch schon die Antwort darauf, wie oft Mods im jeweiligen Datensatz zensieren mussten.
</div>

In [2]:
#Allgemein die Häufigkeit, wie oft in einem Datensatz zensiert wurde
amount_of_censorship = mod_dataframe.count()
print(amount_of_censorship)

3


In [3]:
#Anzahl der Mods, die im Datensatz aktiv waren (Duplikate wurden entfernt)
amount_of_mods = mod_dataframe.drop('x1').drop('y1').drop('x2').drop('y2').drop('t').drop('pixel_color')
amount_of_mods = amount_of_mods.distinct().count()
print(amount_of_mods)

3


<div style="color:#246fd6">
Beantwortung der 2. Frage:
Betrachtung der zensierten Pixel pro Mod.

D.h. hier muss jeder Mod einzeln betrachtet werden und dabei noch berechnet werden, wie viele Pixel dabei das Quadrat aufgespannt haben.

Berechnung der Pixel: 
x1,y1 => obere linke Ecke des Quadrates 
x2,y2 => untere rechte Ecke des Quadrates

Damit ergibt sich:
```
 x1,y1----------------------x2,y1
      |                    |
      |                    |
      |                    |
    a |                    |
      |                    |
      |                    |
      |                    |
 x1,y2----------------------x2,y2
                b
```
Dann ergibt a * b die Anzahl an Pixel, die dabei berührt wurden und somit auch die Anzahl der Pixel, die dabei zensiert wurden.

a = y2 - y1

b = x2 - x1

Anzahl der zensierten Pixel pro Eintrag = (y2 - y1) * (x2 - x1)
</div>

In [4]:
#Vorbereitung um Anzahl der zensierten Pixel pro Mod zu bestimmen
pixel_amount = mod_dataframe.withColumn("amount",(mod_dataframe.y2 - mod_dataframe.y1) * (mod_dataframe.x2 - mod_dataframe.x1))
#pixel_amount = pixel_amount.orderBy('x','y')
pixel_amount.printSchema()
pixel_amount.show()

root
 |-- user_id: string (nullable = true)
 |-- x1: integer (nullable = true)
 |-- y1: integer (nullable = true)
 |-- x2: integer (nullable = true)
 |-- y2: integer (nullable = true)
 |-- t: long (nullable = true)
 |-- pixel_color: string (nullable = true)
 |-- amount: integer (nullable = true)



+--------------------+---+----+---+----+----------+-----------+------+
|             user_id| x1|  y1| x2|  y2|         t|pixel_color|amount|
+--------------------+---+----+---+----+----------+-----------+------+
|gXOuxCyb/dslyRb5G...| 23|1523|172|1792|1649045540|    #000000| 40081|
|Tqn87vyPxAHtPMu4x...| 51|1691|154|1807|1649046581|    #000000| 11948|
|oPv1iZrj4UMjfhzOT...| 44|1652|165|1899|1649045770|    #000000| 29887|
+--------------------+---+----+---+----+----------+-----------+------+



In [5]:
#Anzahl der zensierten Pixel pro Mod zusammengefasst würden jeden Mod
amount_per_mod = pixel_amount.groupBy('user_id').sum('amount')
amount_per_mod.show()

+--------------------+-----------+
|             user_id|sum(amount)|
+--------------------+-----------+
|gXOuxCyb/dslyRb5G...|      40081|
|oPv1iZrj4UMjfhzOT...|      29887|
|Tqn87vyPxAHtPMu4x...|      11948|
+--------------------+-----------+



In [6]:
#Wie viele Pixel im Durchschnitt pro Mod platziert wurden
average_censors = amount_per_mod.agg({"sum(amount)": "avg"}).show()

+------------------+
|  avg(sum(amount))|
+------------------+
|27305.333333333332|
+------------------+



<div style="color:#246fd6">
Nochmal über alle Mods summiert:
</div>

In [7]:
#Anzahl der insgesamt zensierten Pixel im jeweiligen Datensatz über alle Mods summiert
amounts = pixel_amount.agg({'amount': 'sum'}).show()

+-----------+
|sum(amount)|
+-----------+
|      81916|
+-----------+



Die Anzahl an zensierten Pixeln im Verhältnis zu den überhaupt gesetzten Pixel setzten:

In [8]:
#Im Verhältnis dazu bestimmen, wie viele Pixel von "normalen Nutzern" dagegen platziert wurden im Datensatz?
from src.data.dataset_functions import get_dataframe

dataFrame = get_dataframe(True)
dataFrame.printSchema()

providing ../data/raw/2022_place_canvas_history-000000000004.csv ...
../data/raw/2022_place_canvas_history-000000000004.csv is already in data/raw
root
 |-- user_id: string (nullable = true)
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- t: long (nullable = true)
 |-- pixel_color: string (nullable = true)



In [9]:
#Um herauszufinden, wie viele Pixel insgesamt platziert wurden, müssen nur alle Zeilen gezählt werden, 
# da eine Zeile = 1 platzierter Pixel
amount_placed_pixel = dataFrame.count()
print(amount_placed_pixel)

32772070


In [18]:
#Jetzt ins Verhältnis mit Gesamtanzahl an zensierten Pixeln setzen
amount_total_censor = pixel_amount.agg({'amount': 'sum'}).collect()
for row in amount_total_censor:
    i, = list(row)
    relation = ( i / amount_placed_pixel) * 100
print(relation) #in Porzent angegeben


0.24995674670535004
